In [ ]:
#pull terms from sql ; cant concat becuase the rows not being lists will f**k you up in memory 
#name onto for matrix and output purposes 
#put each key in a dict 
    #label2ilx ... ilx2row
#parse for each key in ontologies based on accepted predicates
    #label2iri ... iri2row
# each mach 
#   ilx_iri, ilx_label, ilx_definition, ilx_synonym, ilx_superclass, ilx_type, ilx_existing_ids ... same but <Onto>Iri ... ... has_matching_label has_matching_definition ...
# 0 ...
# 1 ...

In [1]:
from collections import defaultdict
from ilxutils.ilx_pred_map import IlxPredMap
from ilxutils.interlex_sql import IlxSql
from ilxutils.mydifflib import ratio
from ilxutils.tools import open_pickle, create_pickle
import os
import pandas as pd
from pathlib import Path 
from sys import exit

ipm = IlxPredMap()
sql = IlxSql(db_url=os.environ.get('SCICRUNCH_DB_URL_PRODUCTION'))

In [15]:
%time terms = sql.get_terms()

CPU times: user 6.51 s, sys: 560 ms, total: 7.07 s
Wall time: 22 s


In [27]:
%time annos = sql.get_annotations()

CPU times: user 23.2 s, sys: 3.01 s, total: 26.2 s
Wall time: 1min 3s


In [23]:
%time synonyms = sql.get_synonyms()

CPU times: user 2.51 s, sys: 276 ms, total: 2.78 s
Wall time: 3.25 s


In [24]:
%time superclasses = sql.get_superclasses()

CPU times: user 2.86 s, sys: 299 ms, total: 3.16 s
Wall time: 3.59 s


In [25]:
%time relationships = sql.get_relationships()

CPU times: user 130 ms, sys: 12.1 ms, total: 142 ms
Wall time: 178 ms


In [25]:
%time existing_ids = sql.get_existing_ids()

CPU times: user 7.62 s, sys: 636 ms, total: 8.26 s
Wall time: 10.3 s


#################################################################################################################

In [3]:
terms.head()

,id,ilx,orig_uid,orig_cid,uid,cid,label,definition,comment,type,version,status,display_superclass,orig_time,time
0,1,ilx_0100000,32290,0,32290,0,(R)N6-phenylisopropyladenosine,,,term,6,0,1,1481226703,1529356493
1,2,ilx_0100001,32290,0,32290,0,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,,term,6,0,1,1481226705,1529356493
2,3,ilx_0100002,32290,0,34142,0,"1,2-Dibromoethane",Organic Compound;Pesticide;Gasoline Additive/C...,,term,8,0,1,1481226707,1535122210
3,4,ilx_0100003,32290,0,32290,0,1-120 Truncated Alpha-Synuclein Drosophila,This transgenic drosophila expresses a form of...,,term,6,0,1,1481226709,1529356493
4,5,ilx_0100004,32290,0,32290,0,1-D extent,A quality inhering in a bearer by virtue of it...,,term,5,0,1,1481226711,1529356493


In [6]:
annos.head()

,id,tid,annotation_tid,value,uid,term_label,term_ilx,annotation_label,annotation_ilx
0,2265164,1,304445,http://neurolex.org/wiki/nifext_5214,0,(R)N6-phenylisopropyladenosine,ilx_0100000,hasDbXref,ilx_0381360
1,2265173,2,304445,http://neurolex.org/wiki/T3D0044,0,"1,2-Dibromo-3-chloropropane",ilx_0100001,hasDbXref,ilx_0381360
2,2265170,3,304445,http://neurolex.org/wiki/T3D0035,0,"1,2-Dibromoethane",ilx_0100002,hasDbXref,ilx_0381360
3,2265171,4,304445,http://neurolex.org/wiki/nlx_organ_090801,0,1-120 Truncated Alpha-Synuclein Drosophila,ilx_0100003,hasDbXref,ilx_0381360
4,2278612,4,304535,PMID:17376994,0,1-120 Truncated Alpha-Synuclein Drosophila,ilx_0100003,literatureCitation,ilx_0381449


In [108]:
synonyms.head()

,id,tid,literal,ilx,label
0,31340,15070,Age,ilx_0115067,Participant Age
1,31341,15070,Patient Age,ilx_0115067,Participant Age
2,31342,15070,Subject Age,ilx_0115067,Participant Age
23085,331422,3729,eletriptan,ilx_0103728,Eletriptan
23086,333931,4511,GABA-A RECEPTOR,ilx_0104510,GABRA1


In [4]:
superclasses.head()

,id,tid,superclass_tid,term_label,term_ilx,superclass_label,superclass_ilx
0,12494,15070,15069,Participant Age,ilx_0115067,National Cancer Institute Common Data Element,ilx_0115066
23078,307260,2483,10842,Connected spatiotemporal region,ilx_0102482,Spatiotemporal region,ilx_0110841
23079,307261,2484,11600,Connected temporal region,ilx_0102483,Temporal region,ilx_0111599
23080,307331,2558,10743,Cord blood stem cell,ilx_0102557,Somatic stem cell,ilx_0110742
23081,307856,3092,2515,Dependent continuant,ilx_0103091,Continuant,ilx_0102514


In [7]:
relationships.head()

,id,term1_tid,term1_ilx,term1_type,term2_tid,term2_ilx,term2_type,relationship_tid,relationship_ilx,relationship_label
0,1,2,ilx_0100001,term,11841,ilx_0111840,term,12785,ilx_0112784,Has role
1,2,3,ilx_0100002,term,11841,ilx_0111840,term,12785,ilx_0112784,Has role
2,3,22,ilx_0100021,term,3721,ilx_0103720,term,12785,ilx_0112784,Has role
3,4,25,ilx_0100024,term,3577,ilx_0103576,term,12785,ilx_0112784,Has role
4,5,27,ilx_0100026,term,6459,ilx_0106458,term,12797,ilx_0112796,Related to


In [109]:
existing_ids.head()

,id,tid,curie,iri,preferred,ilx,type,label,definition,comment
0,2181964,1,NIFEXT:5214,http://uri.neuinfo.org/nif/nifstd/nifext_5214,1,ilx_0100000,term,(R)N6-phenylisopropyladenosine,,
1,2181981,1,ILX:0100000,http://uri.interlex.org/base/ilx_0100000,0,ilx_0100000,term,(R)N6-phenylisopropyladenosine,,
2,2182003,2,T3D:0044,http://t3db.org/toxins/T3D0044,0,ilx_0100001,term,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,
3,2182004,2,ILX:0100001,http://uri.interlex.org/base/ilx_0100001,1,ilx_0100001,term,"1,2-Dibromo-3-chloropropane",Organic Compound;Pesticide;Organochloride;Orga...,
4,2744315,3,T3D:0035,http://t3db.org/toxins/T3D0035,0,ilx_0100002,term,"1,2-Dibromoethane",Organic Compound;Pesticide;Gasoline Additive/C...,


#################################################################################################################

In [2]:
ilx = open_pickle('/home/tmsincomb/Dropbox/ilx.pickle')

In [3]:
mesh = open_pickle('/home/tmsincomb/Dropbox/scidumps/umls/mesh/mesh2018.pickle')

In [28]:
type(mesh)

rdflib.graph.Graph

In [4]:
from rdflib import Graph, RDF

In [5]:
%%time
g = Graph()
ilx_objs = {str(o).lower().strip():True for s,p,o in ilx.triples((None, None, None))}
#mesh_objs = {o for s,p,o in ilx.triples((None, None, None))}
for subj, pred, obj in mesh.triples((None, RDF.type, None)):
    for p, o in mesh.predicate_objects(subject=subj):
        #if set(str(o).lower().strip()) & ilx_objs:
        if ilx_objs.get(str(o).lower().strip()):
            g.add((subj, pred, obj))
            g.add((subj, p, o))
            
g.serialize('/home/tmsincomb/Desktop/same.ttl', format='turtle')

CPU times: user 2min 7s, sys: 227 ms, total: 2min 7s
Wall time: 2min 7s


In [ ]:
g.serialize('/home/tmsincomb/Desktop/same.ttl', format='turtle')

In [8]:
preds = {p for s,p,o in g.triples((None, None, None))}
for pred in preds:
    prefix, namespace, name = g.compute_qname(pred)
    #print(prefix, name)
preds

{rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#abbreviation'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#altLabel'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#annotation'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#casn1_label'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#entryVersion'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#frequency'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#historyNote'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#lexicalTag'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#nlmClassificationNumber'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#note'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#onlineNote'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#prefLabel'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#previousIndexing'),
 rdflib.term.URIRef('http://id.nlm.nih.gov/mesh/vocab#publicMeSHNote'),
 rdflib.term.URIRef('http://i

In [17]:
ilx_label2ilx = sql.get_label2ilx()

In [20]:
%%time
### LABEL
ilx_objs = {str(o).lower().strip():True for s,p,o in ilx.triples((None, None, None))}
query = """
    PREFIX ns1: <http://id.nlm.nih.gov/mesh/vocab#>
    select ?subj ?label ?definition
    where {
        ?subj ns1:prefLabel|rdfs:label ?label .
        OPTIONAL {?subj ns1:note ?definition .}
    } 
"""
label_count = 0
def_count = 0
common_labels = []
for binding in g.query(query):
    subj, label, definition = binding
    ilx_ids = ilx_label2ilx.get(str(label).lower().strip())
    if ilx_ids:
        #print(label, ilx_ids)
        common_labels.append(label)
        label_count += 1
        if definition: 
            print(definition)
            
print('Labels found:', label_count)
print('Labels w/def:', def_count)

structure
structure
structure
structure
structure
structure
structure
structure
structure
structure
structure
Labels found: 6972
Labels w/def: 0
CPU times: user 17.3 s, sys: 64.1 ms, total: 17.3 s
Wall time: 17.3 s


In [22]:
len(common_labels)

6972

In [24]:
len(set(common_labels))

2844

In [28]:
annos.head()

,id,tid,annotation_tid,value,uid,term_label,term_ilx,annotation_label,annotation_ilx
0,2265164,1,304445,http://neurolex.org/wiki/nifext_5214,0,(R)N6-phenylisopropyladenosine,ilx_0100000,hasDbXref,ilx_0381360
1,2265173,2,304445,http://neurolex.org/wiki/T3D0044,0,"1,2-Dibromo-3-chloropropane",ilx_0100001,hasDbXref,ilx_0381360
2,2265170,3,304445,http://neurolex.org/wiki/T3D0035,0,"1,2-Dibromoethane",ilx_0100002,hasDbXref,ilx_0381360
3,2265171,4,304445,http://neurolex.org/wiki/nlx_organ_090801,0,1-120 Truncated Alpha-Synuclein Drosophila,ilx_0100003,hasDbXref,ilx_0381360
4,2278612,4,304535,PMID:17376994,0,1-120 Truncated Alpha-Synuclein Drosophila,ilx_0100003,literatureCitation,ilx_0381449
